In [4]:
import numpy as np
import os
import sys
import xmltodict
import cv2
import pprint
import shutil
from PIL import Image

In [ ]:
with open("./images/annotations.xml") as f:
    dataset = xmltodict.parse(f.read())
    annotations = dataset['annotations']['image']
    pprint.pp(annotations)

In [ ]:
def load_annotations(data_path):
    with open(os.path.join(data_path, "annotations.xml")) as f:
        dataset = xmltodict.parse(f.read())
    return dataset

def extract_data_from_cvat_annotations(data_path):
    dataset = load_annotations(data_path)
    number_of_images = len(dataset['annotations']['image'])
    
    DATA_DICT = dict()
    task_id_offset = 386
    for INDEX in range(number_of_images):
        annotations = dataset['annotations']['image'][INDEX]
        classes_list = []
        image_name = annotations['@name']
        task_id = int(annotations['@task_id']) - task_id_offset
        img = cv2.imread(os.path.join(data_path, f"task_{task_id}/data", image_name))
        mask = np.zeros((img.shape[0],img.shape[1]))
        try:
            image_boxes_data = annotations['box']
        except:
            image_boxes_data = []
            
        # checks if there is any bounding box in the frame; otherwise it returns an empty mask for that frame without any class names
        frame_data_dict= dict()
        if len(image_boxes_data) == 0:
            frame_data_dict.update({
                "task_id": task_id,
                "image_name": image_name,
                "classes": []
            })
        else:
            labels = []
            for j in range(len(image_boxes_data)):
                try:
                    if image_boxes_data[j]['@label'] != 'Rock' and image_boxes_data[j]['@label'] != 'Unknown':
                        labels.append(image_boxes_data[j]["attribute"]['#text'])
                except:
                    pass
                
            frame_data_dict.update({
                "task_id": task_id,
                "image_name": image_name,
                "classes": labels
            })
        DATA_DICT.update({INDEX : frame_data_dict})
    return DATA_DICT


data_path = "images"
dataset_dict = extract_data_from_cvat_annotations(data_path)
dataset_dict

In [ ]:
filenames = os.listdir('./amphiroa_anceps/output/')
for filename in filenames:
    print(filename)
    image_id = filename[-7:-4]
    # print(image_id)
    
    src = f'./images/full/Deepwatercove_{image_id}.png'
    dst = './amphiroa_anceps/input/'
    shutil.copy(src, dst)
    
    os.rename(f'./amphiroa_anceps/input/Deepwatercove_{image_id}.png', f'./amphiroa_anceps/input/Amphiroa anceps_Deepwatercove_{image_id}.png')

In [35]:
task_id = 5
filenames = os.listdir(f'./images/task_{task_id}/data/')
for filename in filenames:
    image_id = filename[-7:-4]
    # print(filename, image_id)
    
    if os.path.isfile(f'./segmented_masks_anthothoe_albocinta/Anthothoe albocinta_Deepwatercove_{image_id}.png'):
        src = f'./segmented_masks_anthothoe_albocinta/Anthothoe albocinta_Deepwatercove_{image_id}.png'
        dst = './anthothoe_albocinta/output/'
        shutil.copy(src, dst)
        
        src = f'./images/task_{task_id}/data/{filename}'
        dst = './anthothoe_albocinta/input/'
        shutil.copy(src, dst)
        os.rename(f'./anthothoe_albocinta/input/{filename}', f'./anthothoe_albocinta/input/Anthothoe albocinta_Deepwatercove_{image_id}.png')
        print(f"Processed image: {filename}")

Processed image: Deepwatercove_358.png
Processed image: Deepwatercove_390.png
Processed image: Deepwatercove_398.png
Processed image: Deepwatercove_406.png


In [36]:
filenames = os.listdir(f'./anthothoe_albocinta/input/')
for filename in filenames:
    img = cv2.imread(f'./anthothoe_albocinta/input/{filename}', cv2.IMREAD_UNCHANGED)
    flipped = cv2.flip(img, 1)
    
    cv2.imwrite(f'./anthothoe_albocinta/input/flipped_{filename}', flipped)
    
    img = cv2.imread(f'./anthothoe_albocinta/output/{filename}', cv2.IMREAD_GRAYSCALE)
    flipped = cv2.flip(img, 1)
    
    cv2.imwrite(f'./anthothoe_albocinta/output/flipped_{filename}', flipped)
    print(f"Processed image: {filename}")

Processed image: Anthothoe albocinta_Deepwatercove_065.png
Processed image: Anthothoe albocinta_Deepwatercove_108.png
Processed image: Anthothoe albocinta_Deepwatercove_237.png
Processed image: Anthothoe albocinta_Deepwatercove_252.png
Processed image: Anthothoe albocinta_Deepwatercove_273.png
Processed image: Anthothoe albocinta_Deepwatercove_274.png
Processed image: Anthothoe albocinta_Deepwatercove_348.png
Processed image: Anthothoe albocinta_Deepwatercove_349.png
Processed image: Anthothoe albocinta_Deepwatercove_353.png
Processed image: Anthothoe albocinta_Deepwatercove_356.png
Processed image: Anthothoe albocinta_Deepwatercove_357.png
Processed image: Anthothoe albocinta_Deepwatercove_358.png
Processed image: Anthothoe albocinta_Deepwatercove_361.png
Processed image: Anthothoe albocinta_Deepwatercove_364.png
Processed image: Anthothoe albocinta_Deepwatercove_369.png
Processed image: Anthothoe albocinta_Deepwatercove_371.png
Processed image: Anthothoe albocinta_Deepwatercove_372.p

In [6]:
def combine_grayscale_images(img1, img2):
    # Convert images to NumPy arrays
    arr1 = np.array(img1)
    arr2 = np.array(img2)
    
    # Create a mask where pixels in arr2 are not black (non-zero)
    mask = arr2 != 0
    
    # Overwrite pixels in arr1 with pixels from arr2 where mask is True
    arr1[mask] = arr2[mask]
    
    # Convert the result back to an image and save it
    result_img = Image.fromarray(arr1, mode="L")
    result_img = np.array(result_img)
    return result_img


filenames = os.listdir('./predictions/rock/')
for filename in filenames:
    rock_mask = cv2.imread(f'./predictions/rock/{filename}', cv2.IMREAD_GRAYSCALE)
    anthothoe_albocinta_mask = cv2.imread(f'./predictions/anthothoe_albocinta/{filename}', cv2.IMREAD_GRAYSCALE)
    
    anthothoe_albocinta_mask[anthothoe_albocinta_mask == 255] = 127
    merged_mask = combine_grayscale_images(rock_mask, anthothoe_albocinta_mask)
    
    merged_mask[merged_mask == 127] = 0
    
    cv2.imwrite(f'./predictions/rock_modified/{filename}', merged_mask)

In [14]:
rock_mask = cv2.imread(f'./ground_truth/rock/Deepwatercove_128.png', cv2.IMREAD_GRAYSCALE)
print(np.unique(rock_mask))

[  0 255]
